In [61]:
import os, sys
import numpy as np
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
from collections import Counter

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances
from time import time

import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches

from sklearn import manifold, datasets

In [85]:
def read_the_data(fname):
    with open(fname) as f:
        raw = f.read().strip()
        
    units = raw.split('\n\n')
    print(len(units), 'units in the data set')
    
    tags_table = []
    for unit in units:
        t_line = []

        for line in unit.split('\n'):
            tag = line.split('\t')[3]
            t_line.append(tag)
            
        if len(t_line) != 7:
            print(len(t_line))
        tags_table.append(t_line)

    return tags_table

In [86]:
def dissimilarity_matrix(data):    
    #подсчет матрицы
    matrix = []
    for i in range(len(data)):
        sim_row = []
        for j in range(len(data)):
            sim_loc = 0
            for k in range(len(data[i])):
                if data[i][k] == data[j][k] and data[i][k] != 'missing':
                    sim_loc += 1
            sim_row.append(1 - sim_loc/9) #len(data[0]))
        matrix.append(sim_row)
    return matrix

In [87]:
def mds_stuff(matrix, ndim):
    matrix = np.array(matrix)
    mds = MDS(n_components=ndim, random_state=42, dissimilarity='precomputed')
    
    pos = mds.fit_transform(matrix)

    print(mds.stress_)
    DE = euclidean_distances(pos)
    stress = 0.5 * np.sum((DE - matrix)**2)

    ## Kruskal's stress (or stress formula 1)
    stress1 = np.sqrt(stress / (0.5 * np.sum(matrix**2)))
    print("Kruskal's Stress :")
    print("[Poor > 0.2 > Fair > 0.1 > Good > 0.05 > Excellent > 0.025 > Perfect > 0.0]")
    print(stress1)
    return pos   

In [94]:
#Рисуем карту

def make_plot(lang_ind, langs, dim_x, dim_y):
    fig = plt.subplots(dpi=700, figsize=(9,8))
    for i in range(len(tags)):
        plt.scatter(pos[i, dim_x], pos[i, dim_y], c='magenta')
        plt.text(pos[i, dim_x], pos[i, dim_y], tags[i][lang_ind], fontsize=6)


    plt.xlabel ('DIMENSION '+str(1))
    plt.ylabel ('DIMENSION '+str(2))
    plt.savefig('pictures/' + ' ' + langs[lang_ind] + '.png')

# if tags[i][0] == 'надо':
#     plt.scatter(pos[i, dim_x], pos[i, dim_y], c='magenta')
# elif tags[i][0] == 'нужно':
#     plt.scatter(pos[i, dim_x], pos[i, dim_y], c='orange')
# elif tags[i][0] == 'приходиться':
#     plt.scatter(pos[i, dim_x], pos[i, dim_y], c='blue')
# else:
#     plt.scatter(pos[i, dim_x], pos[i, dim_y], c='grey')

# if tags[i][lang_ind] == 'have to':
#     plt.scatter(pos[i, dim_x], pos[i, dim_y], c='magenta')
# elif tags[i][lang_ind] == 'need to':
#     plt.scatter(pos[i, dim_x], pos[i, dim_y], c='orange')
# elif tags[i][lang_ind] == 'must':
#     plt.scatter(pos[i, dim_x], pos[i, dim_y], c='blue')
# else:
#     plt.scatter(pos[i, dim_x], pos[i, dim_y], c='grey')

In [89]:
tags = read_the_data('dataset_tagged_LN.tsv')

195 units in the data set


In [96]:
tags[:5]

[['приходиться',
  'have to',
  'müssen',
  'andare',
  'tener que',
  'être à',
  'доведеться'],
 ['imperative',
  'have to',
  'müssen',
  'imperative',
  'imperative',
  'imperative',
  'imperative'],
 ['нет надобности',
  'need to',
  'sei',
  'essere il caso',
  'necesario',
  'missing',
  'мати потреби'],
 ['приходиться',
  'have to',
  'sollen',
  'dovere',
  'tener que',
  'il faut',
  'доведеться'],
 ['приходиться',
  'have to',
  'müssen',
  'bisognare',
  'missing',
  'il faut',
  'доведеться']]

In [90]:
matrix = dissimilarity_matrix(tags)

In [91]:
n_dim = 15
pos = mds_stuff(matrix, n_dim)

40.07628578854518
Kruskal's Stress :
[Poor > 0.2 > Fair > 0.1 > Good > 0.05 > Excellent > 0.025 > Perfect > 0.0]
0.055740385456280676



In [95]:
langs = ['ru', 'en', 'de', 'it', 'es', 'fr', 'uk']
dim_x = 0
dim_y = 1

for i in range(len(langs)):
    make_plot(i, langs, dim_x, dim_y)